# **Large-Scale Kinship Recognition Data Challenge: Kinship Verification STARTER NOTEBOOK**

We provide framework code to get you started on the competition. The notebook is broken up into three main sections. 
1. Data Loading & Visualizing
2. Data Generator & Model Building
3. Training & Testing Model

We have done the majority of the heavy lifting by making the data easily and readily accessible through Google Drive. Furthermore, we have made the task easier by creating a dataloader and fully trained end-to-end model that predicts a binary label (0 or 1) denoting whether two faces share a kinship relation. 

**WARNING: IF YOU HAVE NOT DONE SO**

Change to GPU:

Runtime --> Change Runtime Type --> GPU

Mount to Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


Install Libraries

In [ ]:
# %%capture
# !pip install keras_vggface
# !pip install keras_applications
# !pip install deepface

In [1]:
from collections import defaultdict
from glob import glob
from random import choice, sample

import cv2
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, LayerNormalization, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2

from deepface.commons.functions import find_input_shape, normalize_input
from deepface.DeepFace import build_model

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

2021-08-09 16:26:57.777995: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


train_relationships.csv contains pairs of image paths which are positive samples (related to each other).

train-faces contains the images for training itself.

In [2]:
# Hyperparameters
BASE_MODEL = 'VGG-Face'
IGNORE_TOP_NLAYERS_ARCH = 2 #-11 #6
IGNORE_BOTTOM_NLAYERS_TUNE = -2 #15
IGNORE_TOP_NLAYERS_TUNE = 0
NORMALIZATION = 'base'
BASE_PATH = "/root/KinshipRecognition"

# Modify paths as per your method of saving them
train_file_path = f"{BASE_PATH}/data/aug_train_ds.csv"
train_folders_path = f"{BASE_PATH}/data/train/train-faces/"
# All images belonging to families F09** will be used to create the validation set while training the model
# For final submission, you can add these to the training data as well
val_families = "F06"

# Get input shape and normalization method.
input_shape = find_input_shape(build_model(BASE_MODEL))
if NORMALIZATION not in ['raw', 'Facenet', 'Facenet2018', 'VGGFace', 'VGGFace2', 'ArcFace']:
    NORMALIZATION = 'base' 


2021-08-09 12:39:09.756003: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-09 12:39:09.756783: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-09 12:39:09.818099: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-08-09 12:39:09.818158: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-08-09 12:39:09.821794: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-08-09 12:39:09.821891: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

In [3]:
all_images = glob(train_folders_path + "*/*/*.jpg")

train_images = [x for x in all_images if val_families not in x]
val_images = [x for x in all_images if val_families in x]

train_person_to_images_map = defaultdict(list)

ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

In [4]:
all_images

['/root/KinshipRecognition/data/train/train-faces/F0126/MID2/P01299_face0.jpg',
 '/root/KinshipRecognition/data/train/train-faces/F0126/MID2/P01298_face3.jpg',
 '/root/KinshipRecognition/data/train/train-faces/F0126/MID2/P01306_face1.jpg',
 '/root/KinshipRecognition/data/train/train-faces/F0126/MID2/P01313_face2.jpg',
 '/root/KinshipRecognition/data/train/train-faces/F0126/MID2/P01301_face3.jpg',
 '/root/KinshipRecognition/data/train/train-faces/F0126/MID2/P01309_face2.jpg',
 '/root/KinshipRecognition/data/train/train-faces/F0126/MID2/P01307_face1.jpg',
 '/root/KinshipRecognition/data/train/train-faces/F0126/MID2/P01297_face1.jpg',
 '/root/KinshipRecognition/data/train/train-faces/F0126/MID4/P01303_face1.jpg',
 '/root/KinshipRecognition/data/train/train-faces/F0126/MID4/P01304_face3.jpg',
 '/root/KinshipRecognition/data/train/train-faces/F0126/MID1/P01304_face2.jpg',
 '/root/KinshipRecognition/data/train/train-faces/F0126/MID1/P01308_face2.jpg',
 '/root/KinshipRecognition/data/train/tr

In [5]:
relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values, relationships.relationship.values))
relationships = [(x[0],x[1],x[2]) for x in relationships if x[0][:10] in ppl and x[1][:10] in ppl]

train = [x for x in relationships if val_families not in x[0]]
val = [x for x in relationships if val_families in x[0]]

In [6]:
def read_img(path, input_shape, normalization='base'):
    img = cv2.imread(path, -1)
    img = cv2.resize(img, input_shape)
    img = cv2.normalize(img,  np.zeros(img.shape[:2]), 0, 255, cv2.NORM_MINMAX)
    img = normalize_input(img, normalization=normalization)
    return np.array(img).astype(np.float)

Define a data generator. Here our data generator will generate a batch of examples which will be used by our model in training. It will generate two images, one for each in the pair as well as a label associated with it.

In [7]:
def gen(list_tuples, person_to_images_map, input_shape, batch_size=16, normalization='base'):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size)
        
        # All the samples are taken from train_ds.csv, labels are in the labels column
        labels = []
        for tup in batch_tuples:
            labels.append(tup[2])
        labels = np.array(labels)

        # Original images preprocessed
        X1 = [x[0] for x in batch_tuples]
        X1 = np.array([read_img(train_folders_path + x, input_shape, normalization=NORMALIZATION) for x in X1])
        X2 = [x[1] for x in batch_tuples]
        X2 = np.array([read_img(train_folders_path + x, input_shape, normalization=NORMALIZATION) for x in X2])
        
        # Mirrored images
        X1_mirror = np.asarray([cv2.flip(x, 1) for x in X1])
        X2_mirror = np.asarray([cv2.flip(x, 1) for x in X2])
        X1 = np.r_[X1, X1_mirror]
        X2 = np.r_[X2, X2_mirror]
        
        yield [X1, X2], np.r_[labels, labels]

Here is an ensemble model built with two resnet-50 architectures, pre-trained, with which we can apply transfer leraning on. This model achieves the baseline and the goal is to expand on this work. There have been papers exploring different architectures as well as introducing BatchNormalization among many other techniques to improve how well the model recognizes kinship between two faces.

In [8]:
def baseline_model(model_name, fine_tune=True):
    input_1_og = Input(shape=input_shape + (3,))
    input_2_og = Input(shape=input_shape + (3,))
#     input_1_mir = Input(shape=input_shape + (3,))
#     input_2_mir = Input(shape=input_shape + (3,))

    backbone = Sequential()
    # ORIGINAL BLOCK
    for layer in build_model(model_name).layers[:-IGNORE_TOP_NLAYERS_ARCH]:
        backbone.add(layer)
    for x in backbone.layers:
        x.trainable = False

    if fine_tune:
        for x in backbone.layers[:IGNORE_BOTTOM_NLAYERS_TUNE]:
            x.trainable = False
        if IGNORE_TOP_NLAYERS_TUNE == 0:
            for x in backbone.layers[IGNORE_BOTTOM_NLAYERS_TUNE:]:
                x.trainable = True
        else:
            for x in backbone.layers[IGNORE_BOTTOM_NLAYERS_TUNE:-IGNORE_TOP_NLAYERS_TUNE]:
                x.trainable = True

            
#     for layer in build_model(model_name).layers[:15]:
#         backbone.add(layer)
#     if fine_tune:
#         for x in backbone.layers:
#             x.trainable = True
#     else:
#         for x in backbone.layers:
#             x.trainable = False

    x1 = backbone(input_1_og)
    x2 = backbone(input_2_og)
#     x1_mir = backbone(input_1_mir)
#     x2_mir = backbone(input_2_mir)

    x1 = GlobalAvgPool2D()(x1)
    x2 = GlobalAvgPool2D()(x2)
#     x1_mir = GlobalAvgPool2D()(x1_mir)
#     x2_mir = GlobalAvgPool2D()(x2_mir)

#     x1 = tf.math.reduce_mean(tf.stack([x1_og, x1_mir], axis=0), axis=0)
#     x2 = tf.math.reduce_mean(tf.stack([x2_og, x2_mir], axis=0), axis=0)
    
    x1 = LayerNormalization(axis=-1, epsilon=0.001, center=False, scale=False)(x1)
    x2 = LayerNormalization(axis=-1, epsilon=0.001, center=False, scale=False)(x2)

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])
    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    x4 = Subtract()([x1_, x2_])
    x5 = Multiply()([x1, x2])
    x = Concatenate(axis=-1)([x3, x4, x5])
        
#     x = LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=True)(x)
    x = Dense(16, activation="relu")(x)
    x = Dropout(0.05)(x)    
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.05)(x)    
    x = Dense(16, activation="tanh")(x)
#     x = LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=False)(x)
    x = Dropout(0.05)(x)    
    out = Dense(1, kernel_regularizer=L2(.01), activation="sigmoid")(x)

    model = Model([input_1_og, input_2_og], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00002))

    model.summary()

    return model


Save the best model to your drive after each training epoch so that you can come back to it. ReduceLROnPlateau reduces the learning rate when a metric has stopped improving, in this case the validation accuracy. 

In [9]:
model_out_path = f"{BASE_PATH}/log/vgg_face_sigmoidtest_finetune.h5"

checkpoint = ModelCheckpoint(model_out_path, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)

callbacks_list = [checkpoint, reduce_on_plateau]

model = baseline_model(BASE_MODEL, fine_tune=True)


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 1, 1, 4096)   134260544   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 4096)         0           sequential_1[0][0]         

In [10]:
for l in model.layers[2].layers:
    print(l.name, l.trainable)

zero_padding2d False
conv2d False
zero_padding2d_1 False
conv2d_1 False
max_pooling2d False
zero_padding2d_2 False
conv2d_2 False
zero_padding2d_3 False
conv2d_3 False
max_pooling2d_1 False
zero_padding2d_4 False
conv2d_4 False
zero_padding2d_5 False
conv2d_5 False
zero_padding2d_6 False
conv2d_6 False
max_pooling2d_2 False
zero_padding2d_7 False
conv2d_7 False
zero_padding2d_8 False
conv2d_8 False
zero_padding2d_9 False
conv2d_9 False
max_pooling2d_3 False
zero_padding2d_10 False
conv2d_10 False
zero_padding2d_11 False
conv2d_11 False
zero_padding2d_12 False
conv2d_12 False
max_pooling2d_4 False
conv2d_13 False
dropout False
conv2d_14 True
dropout_1 True


In [11]:
model.fit(gen(train, train_person_to_images_map, input_shape, batch_size=16, normalization=NORMALIZATION), 
          validation_data=gen(val, val_person_to_images_map, input_shape, batch_size=16, normalization=NORMALIZATION), 
          epochs=25, verbose=1, use_multiprocessing=False, workers=1,
          callbacks=callbacks_list, steps_per_epoch=300, validation_steps=200)

Epoch 1/25


2021-08-09 12:39:32.708726: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-09 12:39:32.731290: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2300000000 Hz
2021-08-09 12:39:33.921364: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-08-09 12:39:34.080340: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


300/300 [==============================] - 66s 211ms/step - loss: 0.7541 - acc: 0.5914 - val_loss: 0.6392 - val_acc: 0.6905

Epoch 00001: val_acc improved from -inf to 0.69047, saving model to /root/KinshipRecognition/log/vgg_face_sigmoidtest_finetune.h5
Epoch 2/25
300/300 [==============================] - 62s 208ms/step - loss: 0.6483 - acc: 0.6677 - val_loss: 0.6445 - val_acc: 0.6881

Epoch 00002: val_acc did not improve from 0.69047
Epoch 3/25
300/300 [==============================] - 62s 208ms/step - loss: 0.5739 - acc: 0.7253 - val_loss: 0.6467 - val_acc: 0.6817

Epoch 00003: val_acc did not improve from 0.69047
Epoch 4/25
300/300 [==============================] - 62s 208ms/step - loss: 0.5383 - acc: 0.7587 - val_loss: 0.6716 - val_acc: 0.6687

Epoch 00004: val_acc did not improve from 0.69047
Epoch 5/25
300/300 [==============================] - 62s 208ms/step - loss: 0.5415 - acc: 0.7529 - val_loss: 0.6889 - val_acc: 0.6478

Epoch 00005: val_acc did not improve from 0.69047
E

KeyboardInterrupt: 

In [ ]:
# Modify paths as per your need
test_path = f"{BASE_PATH}/data/test/"

model = baseline_model(BASE_MODEL, fine_tune=True)
model.load_weights(f"{BASE_PATH}/log/vgg_face_sigmoidtest_finetune.h5")

submission = pd.read_csv(f'{BASE_PATH}/data/test_ds.csv')
predictions = []

for i in range(0, len(submission.p1.values), 32):
    X1 = submission.p1.values[i:i+32]
    X1 = np.array([read_img(test_path + x, input_shape, normalization=NORMALIZATION) for x in X1])

    X2 = submission.p2.values[i:i+32]
    X2 = np.array([read_img(test_path + x, input_shape, normalization=NORMALIZATION) for x in X2])

    pred = model.predict([X1, X2]).ravel().tolist()
    # pred = model([X1, X2], training=True).numpy().ravel().tolist()
    predictions += pred

In [ ]:
print(np.sum(np.asarray(predictions) < .5))
print(len(predictions))
for line in predictions:
    print(line)

The final predictions will need to be rounded: EG 0.01 rounded to 0 and 0.78 rounded to 1. The simple .round() function is sufficient as below.

In [ ]:
# predictions = np.asarray(predictions)
# predictions[predictions <= .60] = 0
d = {'index': np.arange(0, 3000, 1), 'label':predictions}
submissionfile = pd.DataFrame(data=d)
submissionfile = submissionfile.round()

In [ ]:
submissionfile.astype("int64").to_csv(f"{BASE_PATH}/log/results/aug_vggface_full_tunetop2_32-128-32_F06_mirrorface.csv", index=False)

In [ ]:
XX = model.input 
YY = model.layers[-2].output
new_model = Model(XX, YY)
out = new_model.predict([X1, X2, X1_mirror, X2_mirror])
print(out)
print(model.layers[-1].weights)

At this point, download the CSV and submit it on Kaggle to score your predictions.


## Detect Face and Rotate Image


In [ ]:
# !pip install deepface

In [ ]:
# import cv2
# from deepface import DeepFace
# import matplotlib.pyplot as plt
# import matplotlib.image as mimg
# import os

In [ ]:
# in_path = "./drive/MyDrive/KinshipRecognitionStarter/test/"
# out_path = "./drive/MyDrive/KinshipRecognitionStarter/new_test/"
# image_list = os.listdir(in_path)
# for x in image_list:
#   img = DeepFace.detectFace(in_path + x, align=True, detector_backend='mtcnn', enforce_detection=False)
#   mimg.imsave(out_path + x, img)

In [ ]:
# import shutil
# in_path = "./drive/MyDrive/KinshipRecognitionStarter/train/train-faces"
# for x in os.listdir(in_path):
#   try:
#     for y in os.listdir(in_path + "/" + x):
#       print(in_path + "/" + x + "/" + y)
#       try:
#         for z in os.listdir(in_path + "/" + x + "/" + y):
#           filepath = in_path + "/" + x + "/" + y + "/" + z
#           img = DeepFace.detectFace(filepath, align=True, detector_backend='mtcnn', enforce_detection=False)
#           mimg.imsave(filepath, img)
#       except NotADirectoryError:
#           continue
#   except NotADirectoryError:
#       continue

